In [1]:
!pip install keras-tuner

  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-py3-none-any.whl size=78941 sha256=afe8335daba7503e0f5745a4a7e95328f8acde2b703cea3bc9f916e8d37d7cf1
  Stored in directory: c:\users\ganesh babu r\appdata\local\pip\cache\wheels\53\3d\c3\160c686bd74a18989843fcd015e8f6954ca8d834fd2ef4658a
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15359 sha256=a19137eecf5d84c2e0dc6dd44c79d3ff3cac59586b6bb868b049cfbf5921c99a
  Stored in directory: c:\users\ganesh babu r\appdata\local\pip\cache\wheels\08\8f\5f\253d0105a55bd84ee61ef0d37dbf70421e61e0cd70cef7c5e1
Successfully built keras-tuner terminaltables


In [4]:
!pip install tensorflow

  Created wheel for wrapt: filename=wrapt-1.12.1-py3-none-any.whl size=19558 sha256=d7738379e23864ed83cecb58c84ab2b079a2c1a28c5026cf1a136ca5c0bf771a
  Stored in directory: c:\users\ganesh babu r\appdata\local\pip\cache\wheels\5f\fd\9e\b6cf5890494cb8ef0b5eaff72e5d55a70fb56316007d6dfe73
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4835 sha256=c9d589d4c6f1dc38ab1bf215fe4a35fa0a7a46c6d8b3fd11b2fb051217e183b3
  Stored in directory: c:\users\ganesh babu r\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built wrapt termcolor
  Attempting uninstall: h5py
    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.11.2
    Uninstalling wrapt-1.11.2:
      Successfully uninstalled wrapt-1.11.2


In [5]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np

In [9]:
fashion_mnist=keras.datasets.fashion_mnist

In [11]:
 (train_images, train_labels), (test_images, test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 2s 0us/step


In [12]:
train_images-train_images/255.0
test_images=test_images/255.0

In [13]:
train_images[0].shape

(28, 28)

In [15]:
train_images= train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape (len(test_images), 28,28,1)

In [32]:
def build_model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters=hp. Int('conv_1_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
            activation='relu',
            input_shape=(28,28,1)
        ),
        keras.layers.Conv2D(
            filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
            kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
            activation='relu'
        ),
        keras.layers.Flatten(), 
        keras.layers.Dense(
            units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
            activation='relu'
        ),
        keras.layers.Dense(10, activation='softmax') #output layer
    ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

In [33]:
from kerastuner import RandomSearch 
from kerastuner.engine.hyperparameters import HyperParameters

In [34]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5, directory='output', project_name="Mnist Fashion")


In [36]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 08m 11s]
val_accuracy: 0.8851666450500488

Best val_accuracy So Far: 0.8851666450500488
Total elapsed time: 01h 06m 04s
INFO:tensorflow:Oracle triggered exit


In [38]:
model=tuner_search.get_best_models(num_models=1)[0]

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        18464     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2359424   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 2,379,818
Trainable params: 2,379,818
Non-trainable params: 0
_________________________________________________________________
